#### Step 3: Model Fine-tuning
In this notebook, you'll fine-tune the Meta Llama 2 7B large language model, deploy the fine-tuned model, and test it's text generation and domain knowledge capabilities. 

Fine-tuning refers to the process of taking a pre-trained language model and retraining it for a different but related task using specific data. This approach is also known as transfer learning, which involves transferring the knowledge learned from one task to another. Large language models (LLMs) like Llama 2 7B are trained on massive amounts of unlabeled data and can be fine-tuned on domain domain datasets, making the model perform better on that specific domain.

Input: A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file.
For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
The number of files under train and validation should equal to one.

- **You'll choose your dataset below based on the domain you've chosen**

Output: A trained model that can be deployed for inference.\
After you've fine-tuned the model, you'll evaluate it with the same input you used in project step 2: model evaluation. 

---

#### Set up

---
Install and import the necessary packages. Restart the kernel after executing the cell below. 

---

In [1]:
!pip install --upgrade sagemaker datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 40.4 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.214.0
    Uninstalling sagemaker-2.214.0:
      Successfully uninstalled sagemaker-2.214.0


Select the model to fine-tune

In [2]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In the cell below, choose the training dataset text for the domain you've chosen and update the code in the cell below:  

To create a finance domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/finance"`

To create a medical domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/medical"`

To create an IT domain expert model: 

- `"training": f"s3://genaiwithawsproject2024/training-datasets/it"`

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3

estimator = JumpStartEstimator(model_id=model_id,  environment={"accept_eula": "true"},instance_type = "ml.g5.2xlarge") 

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

#Fill in the code below with the dataset you want to use from above 
#example: estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/finance"})
estimator.fit({"training": f"s3://genaiwithawsproject2024/training-datasets/it"})

#### Deploy the fine-tuned model
---
Next, we deploy the domain fine-tuned model. We will compare the performance of the fine-tuned and pre-trained model.

---

In [4]:
finetuned_predictor = estimator.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-530
INFO:sagemaker:Creating endpoint-config with name meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-522
INFO:sagemaker:Creating endpoint with name meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-522


---------!

#### Evaluate the pre-trained and fine-tuned model
---
Next, we use the same input from the model evaluation step to evaluate the performance of the fine-tuned model and compare it with the base pre-trained model. 

---

Create a function to print the response from the model

In [5]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response}")
    print("\n==================================\n")

Now we can run the same prompts on the fine-tuned model to evaluate it's domain knowledge.   

**Replace "inputs"** in the next cell with the input to send the model based on the domain you've chosen. 

**For financial domain:**

  "inputs": "Replace with sentence below from text"  
- "The  investment  tests  performed  indicate"
- "the  relative  volume  for  the  long  out  of  the  money  options, indicates"
- "The  results  for  the  short  in  the  money  options"
- "The  results  are  encouraging  for  aggressive  investors"

**For medical domain:** 

  "inputs": "Replace with sentence below from text" 
- "Myeloid neoplasms and acute leukemias derive from"
- "Genomic characterization is essential for"
- "Certain germline disorders may be associated with"
- "In contrast to targeted approaches, genome-wide sequencing"

**For IT domain:** 

  "inputs": "Replace with sentence below from text" 
- "Traditional approaches to data management such as"
- "A second important aspect of ubiquitous computing environments is"
- "because ubiquitous computing is intended to" 
- "outline the key aspects of ubiquitous computing from a data management perspective."

In [7]:
payload = {
    "inputs": "Outline the key aspects of ubiquitous computing from a data management perspective.",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

Outline the key aspects of ubiquitous computing from a data management perspective.
> [{'generated_text': '\nUbiquitous computing is a relatively new field that has been gaining popularity in recent years. It is a term that refers to the use of technology to make computing devices and systems available and accessible to people in everyday life. This technology can be used to make computing devices and systems more convenient, efficient'}]




In [8]:
payload = {
    "inputs": "A second important aspect of ubiquitous computing environments is",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

A second important aspect of ubiquitous computing environments is
> [{'generated_text': ' the need for novel interaction techniques. We have developed a novel interaction technique that enables users to interact with a large number of devices simultaneously. The technique is based on the use of a large-scale graphical display, a high-resolution camera, and a computer vision system. We have also developed a novel user interface'}]




In [9]:
payload = {
    "inputs": "Traditional approaches to data management such as",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

Traditional approaches to data management such as
> [{'generated_text': " relational databases and data warehouses are no longer sufficient to meet the needs of today's businesses.\nThe increasing volume of data, the velocity at which it is generated and the variety of data sources are forcing businesses to rethink how they manage their data.\nThe cloud offers a vi"}]




Do the outputs from the fine-tuned model provide domain-specific insightful and relevant content? You can continue experimenting with the inputs of the model to test it's domain knowledge. 

**Use the output from this notebook to fill out the "model fine-tuning" section of the project documentation report**

**After you've filled out the report, run the cells below to delete the model deployment** 

`IF YOU FAIL TO RUN THE CELLS BELOW YOU WILL RUN OUT OF BUDGET TO COMPLETE THE PROJECT`

In [15]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-530
INFO:sagemaker:Deleting endpoint configuration with name: meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-522
INFO:sagemaker:Deleting endpoint with name: meta-textgeneration-llama-2-7b-2024-03-31-16-13-19-522
